In [ ]:
# by default, the tf.layers.dense() function uses Xavier intialization (with a unform distribution)
# it could be changed to He intialization by using the variance_scaling_intializer()

import tensorflow as tf
he_init = tf.contrib.layers.variance_scaling_initializer()
hidden1 = tf.layers.dense(X, n_hidden1, activation = tf.nn.relu, 
                         kernel_intializer = he_init, name = 'hidden1')

In [ ]:
# using the elu as activation function for a simple neural net
hidden1 = tf.layers.dense(X, n_hidden1, activation = tf.nn.elu, name = 'hidden1')


def leaky_relu(z, name = None):
    return tf.maximum(0.01 *z, z, name = name)
# using the leaky_relu as activation function for a simple neural net
hidden1 = tf.layers.dense(X, n_hidden1, activation = leaky_relu, name = 'hidden1')

In [ ]:
#Implementing Batch Normalization in tensorflow

import tensorflow as tf
n_inputs= 28* 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10


X = tf.placeholder(tf.float32, shape =(None,n_inputs), name = 'X')
training = tf.placeholder_with_default(False, shape = (), name = 'training')

hidden1 = tf.layers.dense(X, n_hidden1, name = 'hidden1')
bn1 = tf.layers.batch_normalization(hidden1, training = training, momentum = 0.9)
bn1_act = tf.nn.elu(bn1)

hidden2 = tf.layers.dense(bn1_act, n_hidden2, name = 'hidden2')
bn2 = tf.layers.batch_normalization(hidden2, training = training, momentum = 0.9)
bn2_act = tf.nn.elu(bn2)

logits_before_bn = tf.layers.dense(bn2_act, n_outputs, name = 'outputs')
logits = tf.layers.batch_normalization(logits_before_bn, tranining, momentum = 0.9)




In [ ]:
from functools import partial

my_batch_norm_layer = partial(tf.layers.batch_normalization, training = training, momentum = 0.9)
hidden1 = tf.layers.dense(X, n_hidden1, name = 'hidden1')
bn1 = tf.layers.batch_normalization(hidden1, training = training, momentum = 0.9)
bn1_act = tf.nn.elu(bn1)

hidden2 = tf.layers.dense(bn1_act, n_hidden2, name = 'hidden2')
bn2 = tf.layers.batch_normalization(hidden2, training = training, momentum = 0.9)
bn2_act = tf.nn.elu(bn2)

logits_before_bn = tf.layers.dense(bn2_act, n_outputs, name = 'outputs')
logits = tf.layers.batch_normalization(logits_before_bn, tranining, momentum = 0.9)


In [ ]:
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)


with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples//batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run([training_op, extra_update_ops],
                    feed_dict= {training: True, X: X_batch, y:y_batch})
            accuracy_val = accuracy.eval(feed_dict = {X: mnist.test.images, 
                                                     y:mnist.test.labels})
            print(epoch, 'test accuracy: ',accuracy_val)
            
            
        save_path = saver.save(sess, './my_model_final.ckpt')
        

In [ ]:
# gradient clipping
#The optimizer's minimize() function takes care of both computing the gradients and applying them,
# so you must instead call the optimizer's compute_gradients() method first, then
# create an operation to clip the gradients using the clip_by_value() function, and 
# finally create an operation to apply the clipped gradients using the optimizer's 
# apply_gradients() method

threshold = 1.0
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
grads_and_vars = optimizer.compute_gradients(loss)
capped_gvs = [(tf.clip_by_value(grad, -threshold, threshold), var) for grad, var in grads_and_vars]
training_op = optimizer.apply_gradients(capped_gvs)

In [ ]:
# transfer learning - Reusing pretrained layers
# implies using existing neural network that accomplishes a similar task to the one
# you are trying to tackle, then just reuse the lower layers of this network: this is
# called transfer learning. It will not only speed up training considerably, but will
# also require much less training data.

In [ ]:
# reusing a tensorflow model
saver = tf.train.import_meta_graph("./my_model_final.ckpt.meta")

X = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name('y:0')
accuracy = tf.get_default_graph().get_tensor_by_name('eval/accuracy:0')
training_op = tf.get_default_graph().get_operation_by_name('GradientDescent')


# if the pretrained model is not well documented, then you will have to explore the
# graph to find the names of the operations you will need.

for op in tf.get_default_graph().get_operation():
    print(op.name)
    
# a way to create a collection containing all the important operations that people 
# will want to get a handle on is:
 for op in (X, y, accuracy, training_op):
        tf.add_to_collection('my_important_ops', op)
        
# this way people who reuse your model will be able to simply write:
X, y, accuracy, training_op = tf.get_collection('my_important_ops')


# we can then restore the model state using the Saver and continue training

with tf.Session() as sess:
    saver.restore(sess, './my_model_final.ckpt')
    [....] # train the model on your own data

# import_meta_graph() will load the entire original graph    